# Puzzle 2

With your help, Sarah was able to call the private investigator that afternoon, and brought them up to speed. The investigator went to the cleaners directly to see if they could get any more information about the unclaimed rug.

While they were out, Sarah said, “I tried cleaning the rug myself, but there was this snail on it that always seemed to leave a trail of slime behind it. I spent a few hours cleaning it, and the next day the slime trail was back.”

When the investigator returned, they said, “Apparently, this cleaner had a special projects program, where they outsourced challenging cleaning projects to industrious contractors. As they’re right across the street from Noah’s, they usually talked about the project over coffee and bagels at Noah’s before handing off the item to be cleaned. The contractors would pick up the tab and expense it, along with their cleaning supplies.

“So this rug was apparently one of those special projects. The claim ticket said ‘2017 spec JD’. ‘2017’ is the year the item was brought in, and ‘JD’ is the initials of the contractor.

“But they stopped outsourcing a few years ago, and don’t have contact information for any of these workers anymore.”

Sarah first seemed hopeless, and then looked at the USB drive you had just put back in her hand. She said, “I know it’s a long shot, but is there any chance you could find their phone number?”

In [23]:
import pandas as pd
from matplotlib import pyplot as plt
import os
from nameparser import HumanName
import datetime as dt

In [15]:
dir = "/Users/jattenberg/Downloads/noahs-csv/"
files = os.listdir(dir)
names = [n.split(".")[0] for n in files]
dataframe_arr = [pd.read_csv(dir+f) for f in files]

dfs = dict(zip(names, dataframe_arr))

### thought 1: who are the customers with initials jd?

In [20]:
def initialer(n):
    hn = HumanName(n)
    return (hn.first[0] + hn.last[0]).lower()

custs = dfs["noahs-customers"]
jd_custs = custs[custs.name.apply(lambda x: initialer(x) == "jd")]
jd_custs

,customerid,name,address,citystatezip,birthdate,phone
443,1444,Jennifer Davis,4663 Park Ave S,"Brooklyn, NY 11232",1940-07-29,315-356-9127
513,1514,Joshua Downs,144 Park Rd N,"McAllen, TX 78557",1992-05-22,682-474-3747
610,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244
741,1742,Jonathan Davis,862 Morris Park Ave,"Bronx, NY 10462",1947-05-08,585-510-8718
753,1754,Jennifer Diaz,135B Rogers Ave,"Brooklyn, NY 11216",1982-05-20,838-499-3728
...,...,...,...,...,...,...
10269,11270,Jeffrey Dyer,124-3 109th Ave,"South Richmond Hill, NY 11419",1964-12-18,914-631-8362
10465,11466,Janet Davis,693A E New York Ave,"Brooklyn, NY 11203",1998-05-20,838-661-9019
10522,11522,Jeffery Davis,139A E 29th St,"Brooklyn, NY 11226",1964-12-19,914-463-3531
10791,11791,Jonathan Davies,7123 Inter-A 6th Ave,"Brooklyn, NY 11209",1961-10-05,838-788-1575


### Thought 2: What were the orders in 2017?

In [25]:
orders = dfs["noahs-orders"]
orders["ordered"] = pd.to_datetime(orders["ordered"])

mask = orders.ordered.dt.year == 2017
matching_orders = orders[mask]
matching_orders

,orderid,customerid,ordered,shipped,items,total
0,1001,4308,2017-01-31 00:32:19,2017-01-31 07:15:00,NaN,25.52
1,1002,11683,2017-01-31 00:58:31,2017-01-31 18:00:00,NaN,35.33
2,1003,5676,2017-01-31 01:34:40,2017-01-31 09:00:00,NaN,30.79
3,1004,3097,2017-01-31 02:31:24,2017-01-31 19:45:00,NaN,77.60
4,1005,10374,2017-01-31 02:46:09,2017-01-31 14:45:00,NaN,109.04
...,...,...,...,...,...,...
33266,34267,9569,2017-12-31 21:56:33,2017-12-31 21:56:33,NaN,12.18
33267,34268,6892,2017-12-31 22:11:10,2017-12-31 22:11:10,NaN,103.14
33268,34269,4486,2017-12-31 22:12:59,2018-01-07 12:15:00,NaN,430.23
33269,34270,8981,2017-12-31 23:02:44,2018-01-07 17:15:00,NaN,22.10


### Thought 3: who are the matches?

In [27]:
custs_with_orders = jd_custs.merge(matching_orders, on="customerid")
custs_with_orders.name.value_counts()

Jerome Davis Jr.             19
Jennifer Daniels             17
Jacob Davis                  15
Jacqueline Maureen Dudley    15
Jennifer Diaz                14
Joseph Davis                 14
John Deleon III              14
Javier Diaz                  13
James Dunn                   12
Julie Melissa Duffy          11
Joe Day                      11
Jodi Duncan                  11
Jose Duarte                  11
James Duncan Jr.             10
Jonathan Davis                7
Joshua Dickerson              7
Jonathon Diaz                 6
Jeremy Davis                  5
James Davidson                3
James Dixon                   2
James Davis                   1
Julia Dixon                   1
Jessica Duncan                1
Jennifer Duarte               1
Jennifer Davis                1
Jeff Diaz                     1
Name: name, dtype: int64

### Thought 4: Who are customers who havent ordered in a while?

In [43]:
most_recent_order = matching_orders.sort_values("ordered", ascending=True).groupby("customerid").last().ordered
most_recent_order

customerid
1004    2017-12-29 21:46:09
1005    2017-12-08 15:18:31
1007    2017-07-08 12:59:14
1008    2017-12-31 14:43:19
1009    2017-04-21 16:45:10
                ...        
12070   2017-09-20 15:06:24
12071   2017-12-14 09:21:29
12073   2017-12-21 12:55:26
12076   2017-12-26 13:33:49
12077   2017-12-27 10:28:46
Name: ordered, Length: 3875, dtype: datetime64[ns]

In [44]:
with_last_order = custs_with_orders.join(most_recent_order, on="customerid", rsuffix="_last")

In [55]:
with_last_order

,customerid,name,address,citystatezip,birthdate,phone,orderid,ordered,shipped,items,total,ordered_last
0,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244,1341,2017-02-03 11:34:48,2017-02-03 18:00:00,NaN,29.72,2017-10-23 18:46:17
1,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244,4008,2017-03-03 10:23:46,2017-03-03 10:23:46,NaN,7.87,2017-10-23 18:46:17
2,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244,4674,2017-03-09 18:17:38,2017-03-13 15:15:00,NaN,3.90,2017-10-23 18:46:17
3,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244,5314,2017-03-16 03:29:20,2017-03-16 14:15:00,NaN,177.37,2017-10-23 18:46:17
4,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244,5852,2017-03-20 22:45:59,2017-03-22 16:30:00,NaN,46.69,2017-10-23 18:46:17
...,...,...,...,...,...,...,...,...,...,...,...,...
218,12053,Joshua Dickerson,901B E Tremont Ave,"Bronx, NY 10460",1966-04-27,315-629-8623,19020,2017-08-01 08:18:36,2017-08-01 08:18:36,NaN,147.41,2017-11-24 09:17:59
219,12053,Joshua Dickerson,901B E Tremont Ave,"Bronx, NY 10460",1966-04-27,315-629-8623,23332,2017-09-13 07:13:47,2017-09-13 15:45:00,NaN,3.81,2017-11-24 09:17:59
220,12053,Joshua Dickerson,901B E Tremont Ave,"Bronx, NY 10460",1966-04-27,315-629-8623,23988,2017-09-19 08:07:25,2017-09-19 14:45:00,NaN,16.86,2017-11-24 09:17:59
221,12053,Joshua Dickerson,901B E Tremont Ave,"Bronx, NY 10460",1966-04-27,315-629-8623,28847,2017-11-07 06:42:46,2017-11-07 13:45:00,NaN,231.28,2017-11-24 09:17:59


### idea 5: coffee?

In [73]:
products = dfs["noahs-products"]
coffee = products[products.desc.apply(lambda x: x.lower().count("coffee, drip") > 0)]

In [75]:
dfs["noahs-orders_items"]

,orderid,sku,qty,unit_price
0,1001,COL0820,1,25.52
1,1002,TOY8907,1,12.92
2,1002,KIT5813,1,7.99
3,1002,KIT3981,2,7.21
4,1003,KIT7098,1,12.53
...,...,...,...,...
427253,215206,KIT7684,1,13.94
427254,215206,KIT9250,1,3.99
427255,215206,HOM2099,1,91.85
427256,215207,PET0148,1,5.30


In [61]:
"Gluten-free Cat Food, Pumpkin & Pumpkin".lower().count("gluten")

1